<a href="https://colab.research.google.com/github/Katsushi-Nakanishi/quant-macro/blob/main/Midterm_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

pwt1001 = pd.read_stata('/content/drive/MyDrive/Colab Notebooks/Quant_Macro/pwt1001.dta')
# pwtのpageからpwt1001をdowload

In [ ]:
['Australia', 'Austria', 'Belgium', 'Canada', 'Denmark', 'Finland', 'France', 'Germany', 'Greece', 'Iceland', 'Italy', 'Japan', 'Netherlands', 'New Zealand', 'Norway', 'Portugal', 'Spain', 'Sweden', 'Switzerland', 'United Kingdom', 'United States']

['Australia',
 'Austria',
 'Belgium',
 'Canada',
 'Denmark',
 'Finland',
 'France',
 'Germany',
 'Greece',
 'Iceland',
 'Italy',
 'Japan',
 'Netherlands',
 'New Zealand',
 'Norway',
 'Portugal',
 'Spain',
 'Sweden',
 'Switzerland',
 'United Kingdom',
 'United States']

**・Lを総労働時間とした場合**

In [ ]:
#対象の国をリスト化
target_countries = ['Australia', 'Austria', 'Belgium', 'Canada', 'Denmark', 'Finland', 'France', 'Germany', 'Greece', 'Iceland', 'Italy', 'Japan', 'Netherlands', 'New Zealand', 'Norway', 'Portugal', 'Spain', 'Sweden', 'Switzerland', 'United Kingdom', 'United States']

#リストに該当する国の、1990~2019期間のデータを抽出
data = pwt1001[
    pwt1001['country'].isin(target_countries) &
    pwt1001['year'].between(1990, 2019)
]

#必要な変数だけをリスト化し、データを絞る
relevant_cols = ['countrycode', 'country', 'year', 'rgdpna', 'rkna', 'pop', 'emp', 'avh', 'labsh',]
data = data[relevant_cols].dropna()


data['alpha'] = 1 - data['labsh']  # 資本所得分配率（α）= 1 - 労働所得分配率
data['L'] = data['emp'] * data['avh']  # L（総労働時間） = emp * avh
data['y'] = data['rgdpna'] / data['L']  # y = Y/L
data['k'] = data['rkna'] / data['L']  # k = K/L


data = data.sort_values('year')  # 年順にソート
data = data.groupby('countrycode').apply(lambda x:x.assign(
    y_shifted = 100 * x['y'] / x['y'].iloc[0],
    k_shifted = 100 * x['k'] / x['k'].iloc[0]
)).reset_index(drop=True).dropna()  # 国別に、最初の年で基準化したy,k列をデータに追加


def calg(country_data):  #国別のデータに対して、成長会計をおこなう関数を定義

    start_year_actual = country_data['year'].min()  # 初年度
    end_year_actual = country_data['year'].max()  # 終年度

    start_data = country_data[country_data['year'] == start_year_actual].iloc[0]
    end_data = country_data[country_data['year'] == end_year_actual].iloc[0]

    years = end_data['year'] - start_data['year']

    g_y = ((end_data['y_shifted'] / start_data['y_shifted']) ** (1/years) - 1) * 100  # Growth Rate(労働生産性の伸び)

    g_k = ((end_data['k_shifted'] / start_data['k_shifted']) ** (1/years) - 1) * 100  # kの成長率を計算

    alpha_avg = (start_data['alpha'] + end_data['alpha']) / 2.0  # alphaの平均値を計算

    capital_deepening_contrib = alpha_avg * g_k  # Captal Deepning
    tfp_growth_contrib = g_y - capital_deepening_contrib  # TFP Growth

    tfp_share = tfp_growth_contrib / g_y
    cap_share = capital_deepening_contrib / g_y

    # 計算結果の辞書を返す
    return {
        'Country': start_data['country'],
        'Growth Rate': round(g_y, 2),
        'TFP Growth': round(tfp_growth_contrib, 2),
        'Capital Deepening': round(capital_deepening_contrib, 2),
        'TFP Share': round(tfp_share, 2),
        'Capital Share': round(cap_share, 2)
    }

# 国ごとにcalg関数を適用し、結果を辞書のリストとして作成
results_list = data.groupby('countrycode').apply(calg).dropna().tolist()
results_df = pd.DataFrame(results_list)
results_df

<ipython-input-16-3c65660ea67b>:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = data.groupby('countrycode').apply(lambda x:x.assign(
<ipython-input-16-3c65660ea67b>:61: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  results_list = data.groupby('countrycode').apply(calg).dropna().tolist()


,Country,Growth Rate,TFP Growth,Capital Deepening,TFP Share,Capital Share
0,Australia,1.55,0.67,0.87,0.44,0.56
1,Austria,1.48,0.54,0.93,0.37,0.63
2,Belgium,1.10,0.22,0.89,0.20,0.80
3,Canada,1.19,0.50,0.69,0.42,0.58
4,Switzerland,1.05,0.31,0.75,0.29,0.71
5,Germany,1.51,0.77,0.74,0.51,0.49
6,Denmark,1.51,0.65,0.86,0.43,0.57
7,Spain,0.95,0.12,0.83,0.12,0.88
8,Finland,1.73,0.98,0.75,0.57,0.43
9,France,1.22,0.46,0.76,0.38,0.62
